In [2]:
# ============================================================
# Project 6 — Tool Log CSV Extractor + Threshold Flagging (Colab-ready)
# What it does:
# 1) Generates 5 sample CSV log files (each has 3 process parameters)
# 2) Loads all CSVs into one concatenated pandas DataFrame
# 3) Flags cells where value > threshold (creates *_ALARM columns)
# 4) Adds "alarm" column: 1 if ANY parameter exceeds threshold in a row, else 0
# 5) Prints number of alarm rows + prints the alarm rows
# 6) Saves alarm rows into alarm.xlsx (alarm cells highlighted)
#    and also saves merged_logs.csv
# ============================================================

# ----------------------------
# 0) Install dependencies
# ----------------------------
!pip -q install pandas numpy openpyxl

import os, glob, random
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows

# ----------------------------
# 1) CONFIG (edit as needed)
# ----------------------------
BASE_DIR = "/content/project6_logs"      # where CSV logs live
OUT_DIR  = "/content/project6_output"   # outputs (merged + alarm.xlsx)
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

N_FILES = 5
ROWS_PER_FILE = 25

# Exactly 3 process parameter columns
PARAM_COLS = ["RF_power_W", "ChamberPressure_mTorr", "Temp_C"]

# Flag if value > threshold
THRESHOLDS = {
    "RF_power_W": 750,
    "ChamberPressure_mTorr": 45,
    "Temp_C": 110,
}

RNG_SEED = 7
np.random.seed(RNG_SEED)
random.seed(RNG_SEED)

# ----------------------------
# 2) Generate 5 sample CSV log files
# ----------------------------
def generate_sample_csvs(base_dir: str, n_files: int = 5, rows_per_file: int = 25):
    start_time = datetime(2026, 1, 1, 8, 0, 0)
    paths = []

    for i in range(n_files):
        ts = [start_time + timedelta(minutes=2*j) for j in range(rows_per_file)]
        df = pd.DataFrame({
            "timestamp": ts,
            "tool_run_id": [f"RUN_{i+1:02d}"] * rows_per_file,
            # baselines + noise
            "RF_power_W": np.random.normal(loc=600, scale=80, size=rows_per_file).round(1),
            "ChamberPressure_mTorr": np.random.normal(loc=25, scale=8, size=rows_per_file).round(2),
            "Temp_C": np.random.normal(loc=85, scale=12, size=rows_per_file).round(2),
        })

        # Inject a few above-threshold events per file
        for _ in range(np.random.randint(2, 5)):
            r = np.random.randint(0, rows_per_file)
            col = np.random.choice(PARAM_COLS)
            df.loc[r, col] = float(THRESHOLDS[col]) + float(np.random.uniform(1, 40))

        path = os.path.join(base_dir, f"tool_log_{i+1:02d}.csv")
        df.to_csv(path, index=False)
        paths.append(path)

    print(f"Generated {n_files} sample CSVs in: {base_dir}")
    return paths

# Always generate sample files (keeping everything the same)
_ = generate_sample_csvs(BASE_DIR, N_FILES, ROWS_PER_FILE)

# ----------------------------
# 3) Load CSVs into one concatenated DataFrame
# ----------------------------
csv_files = sorted(glob.glob(os.path.join(BASE_DIR, "*.csv")))
if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {BASE_DIR}")

dfs = []
for f in csv_files:
    d = pd.read_csv(f)
    d["source_file"] = os.path.basename(f)
    dfs.append(d)

logs = pd.concat(dfs, ignore_index=True)

# ----------------------------
# 4) Flag cells above thresholds + add alarm column
# ----------------------------
for col, thr in THRESHOLDS.items():
    logs[f"{col}_ALARM"] = (logs[col] > thr).astype(int)

alarm_flag_cols = [f"{c}_ALARM" for c in THRESHOLDS.keys()]
logs["alarm"] = (logs[alarm_flag_cols].sum(axis=1) > 0).astype(int)

alarms_df = logs[logs["alarm"] == 1].copy()

# ----------------------------
# 5) Print summary + alarm rows
# ----------------------------
num_alarm_rows = int(logs["alarm"].sum())
print(f"\nTotal rows: {len(logs)}")
print(f"Number of alarm rows (alarm=1): {num_alarm_rows}\n")

display_cols = ["timestamp", "tool_run_id"] + PARAM_COLS + ["alarm"] + alarm_flag_cols + ["source_file"]

if num_alarm_rows > 0:
    print("Rows with alarms present:")
    print(alarms_df[display_cols].to_string(index=False))
else:
    print("No alarms found.")

# ----------------------------
# 6) Save alarm list to alarm.xlsx with highlighted alarm cells
# ----------------------------
alarm_xlsx_path = os.path.join(OUT_DIR, "alarm.xlsx")
merged_csv_path = os.path.join(OUT_DIR, "merged_logs.csv")

# Save merged logs
logs.to_csv(merged_csv_path, index=False)

# Create Excel file
wb = Workbook()
ws = wb.active
ws.title = "Alarms"

export_df = alarms_df[display_cols].reset_index(drop=True)

# Write header + rows
for row in dataframe_to_rows(export_df, index=False, header=True):
    ws.append(row)

# Highlight cells where the value is above threshold
fill_red = PatternFill(start_color="FFFF6666", end_color="FFFF6666", fill_type="solid")

# Map header names to Excel column indices
header_cells = list(ws[1])
col_to_idx = {cell.value: idx for idx, cell in enumerate(header_cells, start=1)}

for r in range(2, ws.max_row + 1):
    for col, thr in THRESHOLDS.items():
        cidx = col_to_idx[col]
        val = ws.cell(row=r, column=cidx).value
        try:
            if float(val) > thr:
                ws.cell(row=r, column=cidx).fill = fill_red
        except Exception:
            pass

# Simple auto-width
for col_cells in ws.columns:
    max_len = 0
    col_letter = col_cells[0].column_letter
    for cell in col_cells:
        v = "" if cell.value is None else str(cell.value)
        max_len = max(max_len, len(v))
    ws.column_dimensions[col_letter].width = min(max_len + 2, 45)

wb.save(alarm_xlsx_path)

print(f"\nSaved merged logs: {merged_csv_path}")
print(f"Saved alarm list:  {alarm_xlsx_path}")
print("\nIn Colab: left sidebar → Files → project6_output → download alarm.xlsx")

Generated 5 sample CSVs in: /content/project6_logs

Total rows: 125
Number of alarm rows (alarm=1): 18

Rows with alarms present:
          timestamp tool_run_id  RF_power_W  ChamberPressure_mTorr     Temp_C  alarm  RF_power_W_ALARM  ChamberPressure_mTorr_ALARM  Temp_C_ALARM     source_file
2026-01-01 08:16:00      RUN_01  757.855396              29.280000  83.860000      1                 1                            0             0 tool_log_01.csv
2026-01-01 08:20:00      RUN_01  550.000000              55.240591  80.570000      1                 0                            1             0 tool_log_01.csv
2026-01-01 08:34:00      RUN_01  609.900000              29.540000 126.050243      1                 0                            0             1 tool_log_01.csv
2026-01-01 08:40:00      RUN_01  732.100000              80.166929 102.970000      1                 0                            1             0 tool_log_01.csv
2026-01-01 08:46:00      RUN_01  762.300000              27.